In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazonreviews/test.ft.txt.bz2
/kaggle/input/amazonreviews/train.ft.txt.bz2


In [2]:
import pandas as pd
import numpy as np
import bz2
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

%matplotlib inline

In [3]:
device = "cuda:0"

In [4]:
def labels_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts



In [5]:
train_label, train_text = labels_texts('../input/amazonreviews/train.ft.txt.bz2')
test_label, test_text = labels_texts('../input/amazonreviews/test.ft.txt.bz2')

In [6]:
print(train_label[0])
print(train_text[0])

1
Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^


In [7]:
import re
not_numChar = re.compile(r'[\W]')
no_encode = re.compile(r'[^a-z0-1\s]')
def normalisation(texts):
    norm_text = []
    for word in texts:
        lower = word.lower()
        not_punct = not_numChar.sub(r' ', lower)
        exclude_no_encode = no_encode.sub(r'', not_punct)
        norm_text.append(exclude_no_encode)
    return norm_text

In [8]:
train_text = normalisation(train_text)
test_text = normalisation(test_text)

In [9]:
print(train_text[0])

stuning even for the non gamer  this sound track was beautiful  it paints the senery in your mind so well i would recomend it even to people who hate vid  game music  i have played the game chrono cross but out of all of the games i have ever played it has the best music  it backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras  it would impress anyone who cares to listen    


In [10]:
y_train = np.array(train_label)
y_test = np.array(test_label)

In [11]:
y_test.shape

(400000,)

In [12]:
max_features = 8192
maxlen = 128

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train_text)

word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)

Vocabulary Size : 905946


In [13]:
import os 
with open('amazon_dictionary.txt', 'w') as file:
    for key in word_index.keys():
        file.write(key + " " + str(word_index[key]) + ",")

In [14]:
training_token = tokenizer.texts_to_sequences(train_text)
testing_token = tokenizer.texts_to_sequences(test_text)

In [15]:
x_train = pad_sequences(training_token, maxlen = maxlen, padding = 'post')
x_test = pad_sequences(testing_token, maxlen = maxlen, padding = 'post')

In [16]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
BATCH_SIZE = 50

train_data = TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train))
test_data = TensorDataset(torch.from_numpy(x_test), torch.from_numpy(y_test))

train_loader = DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE, drop_last = True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=BATCH_SIZE, drop_last = True)

In [17]:
class SentimentNet(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentNet, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        #self.conv = nn.Conv1d(embedding_dim, 8, kernel_size=5)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        #self.gru = nn.GRU(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        #print(x.shape)
        if len(x) != 1:
            batch_size = x.size(0)
        else:
            batch_size = 1
        x = x.long()
        embeds = self.embedding(x)

        #conv = self.conv(x)
        
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        #hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
        #             weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden

In [18]:
def model_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [19]:
output_size = 1
embedding_dim = 10
hidden_dim = 32
n_layers = 8

model = SentimentNet(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
print(model_params(model))
model.to(device)
lr=0.01
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

9124261


In [20]:
epochs = 2
counter = 0
print_every = 5000
clip = 5
valid_loss_min = np.Inf

model.train()
for i in range(epochs):
    h = model.init_hidden(BATCH_SIZE)
    
    for inputs, labels in train_loader:
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = criterion(output, labels.float())
        loss.backward()
        #nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if counter%print_every == 0:
            val_h = model.init_hidden(BATCH_SIZE)
            val_losses = []
            model.eval()
            for inp, lab in test_loader:
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = criterion(out, lab.float())
                val_losses.append(val_loss.item())
                
            model.train()
            print("Epoch: {}/{}...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './state_dict.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)

Epoch: 1/2... Step: 5000... Loss: 0.693292... Val Loss: 0.693277
Validation loss decreased (inf --> 0.693277).  Saving model ...
Epoch: 1/2... Step: 10000... Loss: 0.694923... Val Loss: 0.693162
Validation loss decreased (0.693277 --> 0.693162).  Saving model ...
Epoch: 1/2... Step: 15000... Loss: 0.688718... Val Loss: 0.693315
Epoch: 1/2... Step: 20000... Loss: 0.693215... Val Loss: 0.693298
Epoch: 1/2... Step: 25000... Loss: 0.693253... Val Loss: 0.693255
Epoch: 1/2... Step: 30000... Loss: 0.693814... Val Loss: 0.693275
Epoch: 1/2... Step: 35000... Loss: 0.694219... Val Loss: 0.693335
Epoch: 1/2... Step: 40000... Loss: 0.691354... Val Loss: 0.693354
Epoch: 1/2... Step: 45000... Loss: 0.697062... Val Loss: 0.693280
Epoch: 1/2... Step: 50000... Loss: 0.691458... Val Loss: 0.693469
Epoch: 1/2... Step: 55000... Loss: 0.694910... Val Loss: 0.693218
Epoch: 1/2... Step: 60000... Loss: 0.693759... Val Loss: 0.693217
Epoch: 1/2... Step: 65000... Loss: 0.699811... Val Loss: 0.693405
Epoch: 1/2

In [21]:
h = model.init_hidden(1)

In [22]:
sentence = "I love this film, it was so awesome!"
trial = torch.tensor(pad_sequences(tokenizer.texts_to_sequences([sentence]), maxlen = maxlen)).to(device)

model(trial, h)[0]

tensor([0.4927], device='cuda:0', grad_fn=<SelectBackward>)

In [23]:
import torch.onnx
torch.onnx.export(model,               # model being run
                  (trial, (torch.randn(n_layers,1,hidden_dim).to(device), torch.randn(n_layers,1,hidden_dim).to(device))),                   # model input (or a tuple for multiple inputs)
                  "lstm-PT.onnx")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
/opt/conda/lib/python3.7/site-packages/torch/onnx/symbolic_opset9.py:1805: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  "or define the initial states (h0/c0) as inputs of the model. ")
